In [164]:
# import cv2
# import numpy as np
# import time

# # Start processing timer
# start = time.time()

# # Load in the sample image
# sample = cv2.imread('../img/sample-10.png')

# # Convert to grayscale
# grayscale = cv2.cvtColor(sample, cv2.COLOR_BGR2GRAY)

# # Apply Gaussian blurring
# blurred = cv2.GaussianBlur(grayscale, (3, 3), 10)

Investigation of using Sobel edge detection instead of Canny. It is more efficient in our case to apply Gaussian blurring, then use the less intensive Canny edge detection method.

In [165]:
# Apply Sobel edge detection
# sobel_x = cv2.Sobel(blurred, cv2.CV_64F, 1, 0, ksize=3)
# sobel_y = cv2.Sobel(blurred, cv2.CV_64F, 0, 1, ksize=3)
# edges = np.sqrt(sobel_x**2 + sobel_y**2)

# Convert edges to uint8 format
# edges = np.uint8(edges)

# Threshold edges to obtain binary image
# _, edges_binary = cv2.threshold(edges, 120, 255, cv2.THRESH_BINARY)

The binary thresholding is very unstable and not suitable.

In [166]:
# Binary thresholding
# Apply binary thresholding
# Parameters:
#   src: input grayscale image
#   thresh: threshold value
#   maxval: maximum value to use with the binary thresholding type
#   type: thresholding type (cv2.THRESH_BINARY, cv2.THRESH_BINARY_INV, etc.)
# Returns:
#   ret: the threshold value (not used in this example)
#   binary_thresholded_sample: the binary thresholded image
# ret, binary_thresholded_sample = cv2.threshold(grayscale, 150, 255, cv2.THRESH_BINARY)

Initial investigation shows that Canny is ideal for this application.

In [167]:
# # Apply Canny edge detection
# # Parameters:
# #   image: input grayscale image
# #   threshold1: first threshold for the hysteresis procedure
# #   threshold2: second threshold for the hysteresis procedure
# # Returns:
# #   edges: output edge map; single channels 8-bit image, which has the same size as image
# edges = cv2.Canny(equalised, 100, 200)

# # Find contours
# # Parameters:
# #   image: input 8-bit single-channel image
# #   mode: contour retrieval mode (cv2.RETR_EXTERNAL: retrieves only the extreme outer contours)
# #   method: contour approximation method (cv2.CHAIN_APPROX_SIMPLE: compresses horizontal, vertical, and diagonal segments and leaves only their end points)
# # Returns:
# #   contours: a Python list of all the contours in the image. Each individual contour is a Numpy array of (x, y) coordinates of boundary points of the object
# contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# # Draw contours on the original image
# contour_image = np.zeros_like(grayscale)
# cv2.drawContours(contour_image, contours, -1, (255, 255, 255), 2)

Use the Hough circle transform to detect circles within our blurred, grayscale image. Sometimes it might detect a glob coming out the junction as a valid particle. We need to filter these out.

In [168]:
# # Apply Hough Circle Transform
# # Parameters:
# #   image: input image
# #   method: detection method (cv2.HOUGH_GRADIENT)
# #   dp: inverse ratio of the accumulator resolution to the image resolution (1)
# #   minDist: minimum distance between the centers of the detected circles (10)
# #   param1: first method-specific parameter (canny edge detection threshold)
# #   param2: second method-specific parameter (accumulator threshold for the circle centers)
# #   minRadius: minimum circle radius (0)
# #   maxRadius: maximum circle radius (0)
# circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=30,
#                             param1=100, param2=50, minRadius=4, maxRadius=0)

# # If circles are found, draw them
# if circles is not None:
#     circles = np.uint16(np.around(circles))
#     for circle in circles[0, :]:
#         center = (circle[0], circle[1])
#         radius = circle[2]
#         # Draw the circle outline
#         cv2.circle(sample, center, radius, (0, 255, 0), 2)

Display some nice images, eh?


In [169]:
# # Stop the timer and calculate the processing time
# stop = time.time()
# processing_time = stop - start

# # Display the original image
# # cv2.imshow('Original Image', sample)

# # Display the grayscale image
# # cv2.imshow('Grayscale Image', grayscale)

# # Show the edge detection output
# # cv2.imshow("Blurred Image", blurred)

# # Display the image with detected circles
# cv2.imshow('Detected Circles', sample)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

Let's merge all the working functionality, and test it on the sample images

In [170]:
import cv2
import numpy as np
import os
import time

def calculate_image_gradient(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Calculate the Sobel gradients in the x and y directions
    gradient_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    gradient_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    
    # Compute the combined gradient magnitude
    gradient_magnitude = cv2.magnitude(gradient_x, gradient_y)
    
    # Compute the average gradient magnitude
    average_gradient = cv2.mean(gradient_magnitude)[0]
    
    return average_gradient

# Create the output directories if they don't exist
output_dir = '../output/'
grayscale_dir = os.path.join(output_dir, 'grayscale')
blurred_dir = os.path.join(output_dir, 'blurred')
annotated_dir = os.path.join(output_dir, 'annotated')
os.makedirs(grayscale_dir, exist_ok=True)
os.makedirs(blurred_dir, exist_ok=True)
os.makedirs(annotated_dir, exist_ok=True)

# Get the list of input files in the directory
files = os.listdir('../img/')

# Filter the list to keep only the '.png' files
png_files = sorted([file for file in files if file.endswith('.png')])

# Determine the upper limit for the loop
num_images = len(png_files)

# Create a dictionary to store metrics for each image
image_metrics = {}

# Loop through each of the images
for i in range(1, num_images + 1):
    # Start processing timer
    start = time.time()

    # Load the image
    image_path = os.path.join('../img/', f'sample-{i}.png')
    sample = cv2.imread(image_path)
    
    # Calculate 'bluriness' of an image
    gradient = calculate_image_gradient(sample)  
    
    # Convert to grayscale
    grayscale = cv2.cvtColor(sample, cv2.COLOR_BGR2GRAY)
    # Apply Gaussian blurring
    blurred = cv2.GaussianBlur(grayscale, (3, 3), 10)

    # Apply Hough Circle Transform
    # Parameters:
    #   image: input image
    #   method: detection method (cv2.HOUGH_GRADIENT)
    #   dp: inverse ratio of the accumulator resolution to the image resolution (1)
    #   minDist: minimum distance between the centers of the detected circles (10)
    #   param1: first method-specific parameter (canny edge detection threshold)
    #   param2: second method-specific parameter (accumulator threshold for the circle centers)
    #   minRadius: minimum circle radius (0)
    #   maxRadius: maximum circle radius (0)
    circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=40,
                                param1=70, param2=50, minRadius=10, maxRadius=500)

    # Init lists to store circle information
    circle_centers = []
    circle_radii = []

    # If circles are found, draw them
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for circle in circles[0, :]:
            center = (circle[0], circle[1])
            radius = circle[2]
            # Append circle center and radius to lists
            circle_centers.append(center)
            circle_radii.append(radius)
            # Draw the circle outline
            cv2.circle(sample, center, radius, (0, 255, 0), 2)

    # Stop the timer and calculate the processing time
    stop = time.time()
    processing_time = stop - start

    # Gather metrics for this image
    num_circles = len(circle_centers)
    sorted_radii = sorted(circle_radii)

    # Calculate the median circle radius
    if sorted_radii:
        if len(sorted_radii) % 2 == 0:
            median_radius = (sorted_radii[len(sorted_radii) // 2 - 1] + sorted_radii[len(sorted_radii) // 2]) / 2
        else:
            median_radius = sorted_radii[len(sorted_radii) // 2]
    else:
        median_radius = None

    # Record the metrics in the dictionary
    image_metrics[f'sample-{i}.png'] = {
        'Number of circles': num_circles,
        'Circle radii (sorted)': sorted_radii,
        'Median circle radius': median_radius,
        'Processing time': processing_time,
        'Blur Gradient' : gradient
    }

    # Save the grayscale image
    grayscale_output_path = os.path.join(grayscale_dir, f'sample-{i}-grayscale.png')
    cv2.imwrite(grayscale_output_path, grayscale)

    # Save the blurred image
    blurred_output_path = os.path.join(blurred_dir, f'sample-{i}-blurred.png')
    cv2.imwrite(blurred_output_path, blurred)

    # Create a canvas to draw the image and metrics
    canvas_width = int(sample.shape[1] * 1.3)  # Width of the canvas (80% + 20%)
    canvas_height = sample.shape[0]
    canvas = np.zeros((canvas_height, canvas_width, 3), dtype=np.uint8)

    # Draw the processed sample image on the canvas
    canvas[:, :sample.shape[1]] = sample

    # Define font properties
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.5
    font_color = (255, 255, 255)
    line_height = 20
    x_offset = sample.shape[1] + 10  # Start position for the metrics on the right-hand side

    # Draw the sample name
    cv2.putText(canvas, f"Sample: sample-{i}.png", (x_offset, line_height), font, font_scale, font_color, 1)

    # Draw the metrics
    cv2.putText(canvas, f"Number of circles: {num_circles}", (x_offset, 2 * line_height), font, font_scale, font_color, 1)
    cv2.putText(canvas, f"Circle radii (sorted): {sorted_radii}", (x_offset, 3 * line_height), font, font_scale, font_color, 1)
    cv2.putText(canvas, f"Median circle radius: {median_radius}", (x_offset, 4 * line_height), font, font_scale, font_color, 1)
    cv2.putText(canvas, f"Blur gradient: {gradient:.3f}", (x_offset, 5 * line_height), font, font_scale, font_color, 1)
    cv2.putText(canvas, f"Processing time: {processing_time:.3f} seconds", (x_offset, 6 * line_height), font, font_scale, font_color, 1)

    # Save the annotated image
    annotated_output_path = os.path.join(annotated_dir, f'sample-{i}-annotated.png')
    cv2.imwrite(annotated_output_path, canvas)

    print(f"Processed sample-{i}.png and saved annotated image to {annotated_output_path}")

# Print the recorded metrics in a nice format
print("\n\nImage Metrics:")
for image_name, metrics in image_metrics.items():
    print(f"Metrics for {image_name}:")
    for metric_name, value in metrics.items():
        print(f"{metric_name}: {value}")
    print()  # Add a blank line between images



Processed sample-1.png and saved annotated image to ../output/annotated/sample-1-annotated.png
Processed sample-2.png and saved annotated image to ../output/annotated/sample-2-annotated.png
Processed sample-3.png and saved annotated image to ../output/annotated/sample-3-annotated.png
Processed sample-4.png and saved annotated image to ../output/annotated/sample-4-annotated.png
Processed sample-5.png and saved annotated image to ../output/annotated/sample-5-annotated.png


Processed sample-6.png and saved annotated image to ../output/annotated/sample-6-annotated.png
Processed sample-7.png and saved annotated image to ../output/annotated/sample-7-annotated.png
Processed sample-8.png and saved annotated image to ../output/annotated/sample-8-annotated.png
Processed sample-9.png and saved annotated image to ../output/annotated/sample-9-annotated.png
Processed sample-10.png and saved annotated image to ../output/annotated/sample-10-annotated.png
Processed sample-11.png and saved annotated image to ../output/annotated/sample-11-annotated.png
Processed sample-12.png and saved annotated image to ../output/annotated/sample-12-annotated.png
Processed sample-13.png and saved annotated image to ../output/annotated/sample-13-annotated.png
Processed sample-14.png and saved annotated image to ../output/annotated/sample-14-annotated.png
Processed sample-15.png and saved annotated image to ../output/annotated/sample-15-annotated.png
Processed sample-16.png and saved anno